## Brendan Liu (9441056), Elaine Sieng (4279873), Joshua Kim (9660796)

## Introduction

We have visually explored the data and are working on the actual model fitting and tuning. Most of the analysis section will be focused on the things we are planning to do in the next week.


## Data Overview

We have 5GB of meteorological and water quality data from the National Estuarine Research Reserve System’s System-wide Monitoring Program. Collected from 2004-2006, the data accounts for 26 reserves, some divided into subregions with their own sensors and data. The meteorological data is collected in 15/60 minute intervals over 3 years. The water quality data is collected in 30/60 minute intervals over 3 years. All the data is numeric (more specifically float) and there are some missing data in some of the files. 

The meteorological features include 25 parameters: air temperature (ATemp), relative humidity (RH), barometric pressure (BP), wind speed (WSpd), wind direction (WDir), wind direction standard deviation (SdWDir), maximum wind speed (MaxWSpd) , minimum wind speed (MinWSpd), total precipitation (TotPrcp), total photosynthetically active radiation (TotPAR), and average battery voltage (AvgVolt). Each one of these parameters comes with a min and max value for the recorded time period as well as a timestamp column for when the min or max value was recorded.

We discover that total precipitation (TotPrcp) has too many null values to be considered significant for analysis, these null values arise from the inconsistency of reporting across different regions and time periods. Also, different regions report total precipitation as a cumulative sum while others report hourly total values. An excess in null values and inconsistency of data collection leads to the omittance of TotPrcp in our analysis. 

Other parameters we omit are all the data including timestamps for when the mins and maxes occurred for temperature, relative humidity, barometric pressure, and wind speed (Temp, RH, BP, WSpd). The time at which this information is collected has no direct impact on predicting water quality. 

After removing these parameters, we are left with 17 parameters, 8 of which are the minimum and maximum values of temperature, relative humidity, barometric pressure, and wind speed (Temp, RH, BP, WSpd). In our EDA, we will see whether or not to omit these parameters depending on how correlected these mins and maxes are to the given average values. 

Ideally, we would create localized models for different regions of the nation. However, for the sake of simplicity, we cleaned our data to reflect the national average. We understand that there are limitations to dealing with national versus local inputs. A simple example being temperatures fluctuate more on the east coast rather than the west coast. 

According to DeWeber et al, water temperature is a fundamental feature in sustaining ecosystems, and therefore a valuable property of water quality. We will be using Temp (the temperature of the water at a certain time) as our predictor variable.

## Methods

The raw data is unfit to be passed directly into a machine learning algorithm and must be transformed into a more machine readable format.

Columns to Drop:

TotPrcp, DnIrrad, TotPAAR are all reported inconsistently over the regions and time periods studied. After aggregating the data, these three columns are more than half null. We decided that the best way to deal with the overwhelming null values is to just drop the column in its entirety. Inferring the value would be disastrous since the reason why it isn’t reported is because that particular locality does not record that specific data. Instead of dropping the data column wise, we decided to keep columns that were consistently reported across all 12 regions. This is possible since certain regions have different instrumentation capability. We also drop the columns which indicate when the minimum and maximum value of the feature was recorded. 

Aggregation of Data:

The raw data has just under 1 million data points each year, from 2004-2005. We chose to do mean aggregates of the data on a daily time scale using the measurements taken every 15 minutes. In most of the literature attempting to forecast water temperature, the data used is localized to a specific region. We will investigate if the aggregate national data carries sufficient information to extrapolate to more regional datasets.

Heteroskedasticity:

We investigate the correlation between predictors and their maximum and minimum reported values on a daily time frame. 

Normalization:

The scale of the data varies heavily by feature. Relative humidity (RH) is on the degree of 70-100 while the average wind speed (WSpd) is between 0-5. Normalizing data makes sure that the scale of the data does not influence the importance of the feature when being fitted to a model. 

Standard water temperature regression models (according to Zhu et al.):
We will not be treating the data as a timeseries forecasting problem. As such, we will be scrubbing the dates from the data and treating each data point as if they are uncorrelated in time.
1. Linear Regression Model
A linear regression can provide a simple first-order estimation of the relationship between water temperature and meteorological conditions. Linear Regression coefficients are easily interpretable and understood. This model can be a good jumping off point and give us a good baseline model to judge other model performances.
2. Non-linear Regression Model
Given the vast amount of data collected, there seems a high probability that the normality assumptions needed for the linear model will be violated. This is especially apparent when we plot the distribution of water temperature over each of the full three years. 
Average temperature in 04:
<img src=./images/distplot_NAvg_Temp_04.png>
Average temperature in 05:
<img src=./images/distplot_NAvg_Temp_05.png>
Average temperature in 06:
<img src=./images/distplot_NAvg_Temp_06.png>
There is a heavy spike between 20-25 degree celsius as well as between 5 and 15 degrees, which is very obviously not normal.

Random Forest

The random forest should capture the variability in the different features. Zhu et al. described the random forest being one of the most robust and accurate models for predicting water temperature. 

aNN

aNNs have become more popular in recent years due to advances in GPU technology. We plan on using a deep multilayer feedforward neural network to regress on our training data. Hyperparameters to be tuned are: learning rate, depth of network, max iterations. Stochastic gradient descent will be the backpropagation algorithm of choice.

Model Evaluation

In this paper, we will use the following performance indicators, the correlation coefficient R and the root mean square error (RMSE). We will also be using some random local data as a testing set to see if our model trained on the national average water temperature works on a local scale.

## Analysis - Results and Discussion

From the literature, we are expecting air temperature to be the most important predictor for water temperature. However, we want to investigate the effects of additional predictors, such as windspeed, barometric pressure, wind direction, etc.. on both simple and complex models. We're expecting the same if not increased performance in accuracy versus models trained soley on average air temperature.

Linear regression is a According to Ficklin et al, we expect to see a direct negative correlation between water quality and dissolved oxygen levels, which means we should not take DO_pct and DO_mgl into consideration when predicting water temperature. Since there is direct correlation, our predictions will be overfitted in relation to the dissolved oxygen variables. We plan on showing the literature values of accuracy next to our own accuracy, to see if adding more predictor variables helped in the prediction of water temperature. We also plan on showing the regression line against the response variable, and see how well it tracks the response. 

Random forests are a classic statistical ensemble method proven to help reduce the variance of the predictions. In addition, random forests can effectively handle missing data, which will prove to be useful since our dataset includes some inconsistencies in data collection between the various regions. We will also be using random forests due to its speed and simplicity, along with the fact that there would be minimal to no need for any tuning of the hyperparameters. The use of random forests will act as a supplement to our main algorithm, which is the artificial neural network. 

Considering the popularity of using the aNN method for predicting water quality parameters, we will also be using it for our data. Similar to DeWeber’s analysis, we will be focusing the majority of our attention on the daily data because we are looking for a general trend throughout multiple locations rather than specific details and trends of one location. Given how the aNN method is generally used for modeling nonlinear relationships, as well as handling interactions between predictor variables, we can expect this method to be the most effective of our analysis. In order to create a more accurate model, we will need to finely tune our parameters, which can be taken from our stochastic gradient descent. After performing k-fold cross validation, we will find the optimal number of neurons for each model. However, we must take into account that it is very easy to underfit or overfit our model, so we will perform multiple tests to find the most accurate result. After doing so, we can expect to have more accurate predictions of water temperature, based not only on air temperature but also the rest of the predictor variables that we decided were necessary. 

## Work in Progress

We wanted to add in this section, because we met on 5/27 to talk about and work on this project and came across a realization. We have over aggregated our data, and much of the features that we have talked about so far now contain little to no meaning. Since we aggregated the data on the national level, things like Barometric pressure, windspeed, wind direction, and even average temperature makes no sense. To put it differently, how would the average national wind speed matter in a prediction of national average water temperature? Going forward, we will be working with a localized dataset, and we have already selected a few neighboring regions that we will be working with. The methods and analysis will all be the same, just with a dataset spanning a few regions, and not across the entire country. We wanted to add this section in case when you read the methods and analysis, you came across the same realization. 

## References

DeWeber & Wagner (2014) DeWeber JT, Wagner T. A regional neural network ensemble for predicting mean daily river water temperature. Journal of Hydrology. 2014;517:187–200. doi: 10.1016/j.jhydrol.2014.05.035.

Ficklin, Stewart & Maurer (2013) Ficklin DL, Stewart IT, Maurer EP. Effects of climate change on stream temperature, dissolved oxygen, and sediment concentration in the Sierra Nevada in California. Water Resources Research. 2013;49:2765–2782. doi: 10.1002/wrcr.20248.

Zhu S, Nyarko EK, Hadzima-Nyarko M. Modelling daily water temperature from air temperature for the Missouri River. PeerJ. 2018;6:e4894. Published 2018 Jun 7. doi:10.7717/peerj.4894